# II. Min hashing

In [1]:
import pandas as pd
import numpy as np
import collections
import itertools
import random
from tqdm import tqdm
from math import floor, sqrt
import ast

Cargar datos preprocesados

In [4]:
resumenx = pd.read_csv('preprocesado.csv', converters={"shingles": ast.literal_eval})

Cargar vocabulario de shingles previamente computado y mostrar su largo

In [5]:
with open('shingles_sample.txt','r') as f:
   shingles_sample = ast.literal_eval(f.read())
un = shingles_sample
len(un)

9958

Esta parte simplemente la usé para debuggear, pero no borrar

In [6]:
resumen = resumenx
resumen

,Unnamed: 0,id,screen_name,text,clean_text,shingles
0,7,1512188055788027904,Rob92029357,@sokio @berfontaine No tienes como revatir alg...,@sokio @berfontaine revatir realidad expuesta ...,"(sabe , ontai, pers, ad ex, kio @, ona s, rev..."
1,20,1512193611575799812,AlejoLatigo,@Hugo_Gutierrez_ Yaaaaaa yyyyyyyyy????????????...,@Hugo_Gutierrez_ Yaaaaaa yyyyyyyyy????????????...,"(? #RE, ?? #R, @Ser, ??? #, yy???, aaa y, yyy..."
2,29,1512198930657955842,GordoUC_,@Molines01 @tere_marinovic Te están mostrando ...,@Molines01 @tere_marinovic estn mostrando rech...,"(echa., nes01, den e, o rec, mostr, oline, ari..."
3,30,1512199583794835467,Orellana21Marce,@mdaza_abogado @Jess99041498 Es que la derecha...,@mdaza_abogado @Jess99041498 derecha quiere em...,"(o @Je, @Jes, impo, quie, 8 der, n ind, @md..."
4,50,1512212224613359617,PATRIOTCHILENO,@tere_marinovic @Sebasti45781359 Orsoni cero a...,@tere_marinovic @Sebasti45781359 Orsoni cero a...,"(45781, as to, @Mait, 81359, ere_m, Maite, ovi..."
...,...,...,...,...,...,...
203721,1266590,1526783486702411776,MaraTer03135126,@BSepulvedaHales aparacistes loca populista #R...,@BSepulvedaHales aparacistes loca populista #R...,"(hazoE, s apa, ista , co/JL, ho ht, Mamar, o h..."
203722,1266611,1527138136060334080,cuncunga1,@mcubillossigall Aqui no hubo bandera ni canci...,@mcubillossigall Aqui bandera cancin nacional....,"(queda, cubil, l. qu, l Aqu, Aqui, http, lad..."
203723,1266615,1527090757202456576,Erwinpohlagurto,@Hugo_Gutierrez_ Últimamente sospecho de todos...,"@Hugo_Gutierrez_ ltimamente sospecho todos, am...","(ues e, http, ltima, amigu, UKXHt, os ht, se..."
203724,1266621,1525908559673712640,RichardDCorobo,@BottoConstituy1 El poder económico nunca va a...,@BottoConstituy1 poder econmico nunca va quere...,"(nca v, co nu, y1 po, http, LkfWL, uerer, ca ..."


## Crear funciones de hash

Primero hallamos un número primo más grande que el universo de shingles

In [7]:
def is_prime(n):
  for i in range(2,int(sqrt(n))+1):
    if (n%i) == 0:
      return False
  return True

numb = len(un)+1
while True:
    result = is_prime(numb)
    if result:
       break
    numb+=1
print("Primo encontrado: ", numb)

Primo encontrado:  9967


Creamos 20 funciones de hash

In [8]:
def crear_hash(a, b, p, n):
    def f(x):
        return ((a * x + b) % p) % n
    return f

In [9]:
h = []
n = len(un)
p = numb
num_hash = 20
for i in range(num_hash):
    a = random.randint(1,p-1)
    b = random.randint(1,p-1)
    h.append(crear_hash(a,b,p,n))

## Firma de Hash

Calcularemos la firma de hash dividiendo el resumen en 100 chunks.

In [10]:
sorted_union = sorted(un)
def chunk_signature(chunk):
    n = len(chunk)
    FH = np.full((num_hash, n), n+100)
    ### Iterar por elementos del vocabulario
    for e in tqdm(range(0,len(sorted_union))): # |union_shingles| = 9967
       
        ### iteramos cada columna, correspondiente a la cantidad de textos (tweets en nuestro caso)  
        for i in range(0,n): # |cols| = 208.139

            ### si el elemento e no está en el texto i, no hacemos nada
            if sorted_union[e] not in chunk.shingles[i]:
                pass
            
            ### si el elemento si está, computamos los valores de h(e) y 
            else:
                for j in range(0,num_hash): # j es función de hash |hash| = 20, filas de la matriz
                    FH[j][i] = min(FH[j][i],h[j](e))
    return FH

Calcular la firma de hash de cada chunk e ir escribiendo a disco

In [11]:
# n_of_chunks = 10
# chunk_size = floor(len(resumen)/n_of_chunks)
# chunk = resumen.iloc[:chunk_size,:]
# FH = chunk_signature(chunk)
# np.savetxt("output/file0.csv", FH, delimiter=",")
# del FH
# for i in tqdm(range(1, n_of_chunks)):
#     if i != (n_of_chunks-1):
#         chunk = resumen.iloc[chunk_size*i:chunk_size*(i+1),:]
#     else:
#         chunk = resumen.iloc[chunk_size*i:,:]
#     chunk = chunk.reset_index()
#     FH = chunk_signature(chunk)
#     np.savetxt(f"output/file{i}.csv", FH, delimiter=",")
#     del FH